#Contexto

En este proyecto se tienen varias fuentes de datos, entre ellas está la base de datos de Google Maps en la cual se tiene información relacionada con diferentes rubros. En este caso nos centraremos en los negocios relacionados con la hotelería, restaurantes y el turismo. Para ello contamos con un archivo que contiene información sobre los negocios en Estados Unidos y una carpeta con los reviews por cada estado.

Dada la cantidad de datos que tiene cada carpeta, se hace necesario filtrar por los rubros de nuestro interés. Lo primero que hacemos es conectarnos a la fuente de datos desde Google Drive y extraer la carpeta `metadata-sitios`

In [ ]:
from collections import OrderedDict
import numpy as np
import pandas as pd
import requests
import json
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ruta_carpeta = '/content/drive/MyDrive/Colab Notebooks/Google Maps/metadata-sitios'

Poteriormente creamos las funciones necesarias para leer y filtrar cada archivo por categoría que esté contenido dentro de la carpeta.

In [ ]:
def leer_archivo(ruta_archivo):

  with open(ruta_archivo, 'r') as file:
    contenido = [json.loads(linea) for linea in file]

  return contenido

In [ ]:
def filtrar_archivo(archivo):

  lista_filtrada = []
  for registro in archivo:
      if isinstance(registro['category'], list):
          contador = 0
          for palabra in registro['category']:
              if palabra.find('restaurant') != -1 or palabra.find('Restaurant') != -1:
                  contador +=1
              elif palabra.find('hotel') != -1 or palabra.find('Hotel') != -1:
                  contador +=1
          if contador >= 1:
            lista_filtrada.append(registro)

  return lista_filtrada

El siguiente codigo itera sobre la carpeta y aplica las funciones de lectura y filtrado devolviendo en la variable `metadata_sitios_final` todos los registros.

In [ ]:
metadata_sitios_final =[]
for archivo in os.listdir(ruta_carpeta):
  ruta_archivo = os.path.join(ruta_carpeta, archivo)
  contenido = leer_archivo(ruta_archivo)
  lista_filtrada = filtrar_archivo(contenido)
  metadata_sitios_final.extend(lista_filtrada)

Una vez tenemos todos los registros en la variable, aplicamos el código a continuación para quedarnos solamente con los registros únicos.

In [ ]:
metadata_sitios_filtrado = list(OrderedDict((str(d), d) for d in metadata_sitios_final).values())

In [ ]:
len(metadata_sitios_filtrado)

218432

De los registros obtenidos eliminamos las claves`MISC` y `hours` debido a que son valores que tienen registros anidados y presentan problemas a la hora de cargar estos datos en Google Cloud Platform.

In [ ]:
metadata_sin_anidado = [{k: v for k, v in registro.items() if k not in ["MISC", "hours"]} for registro in metadata_sitios_filtrado]

Para terminar con los datos de esta variable, la convertimos en un archivo JSONL, el cual es el tipo de archivo que admite Google Cloud Platform.

In [ ]:
# Convierte la lista de objetos JSON en un archivo JSONL

with open('/content/drive/MyDrive/business_google.jsonl', 'w') as jsonl_file:
    for item in metadata_sin_anidado:
        jsonl_file.write(json.dumps(item) + '\n')

In [ ]:
url ='/content/drive/MyDrive/Colab Notebooks/Google_maps_filtrado/business_google.jsonl'

with open(url) as archivo_json:

    business = datos_lista = [json.loads(linea) for linea in archivo_json]

business[0]

{'name': 'San Soo Dang',
 'address': 'San Soo Dang, 761 S Vermont Ave, Los Angeles, CA 90005',
 'gmap_id': '0x80c2c778e3b73d33:0xbdc58662a4a97d49',
 'description': None,
 'latitude': 34.0580917,
 'longitude': -118.2921295,
 'category': ['Korean restaurant'],
 'avg_rating': 4.4,
 'num_of_reviews': 18,
 'price': None,
 'state': 'Open ⋅ Closes 6PM',
 'relative_results': ['0x80c2c78249aba68f:0x35bf16ce61be751d',
  '0x80c2c79998f75fff:0xd7ca5c67e96fb778',
  '0x80c2b899146d7507:0xf4162c12c9cf65f8',
  '0x80c2c77f2d419951:0x26285631b21e324c',
  '0x80c2b8add9016015:0x15836f81a963b35f'],
 'url': 'https://www.google.com/maps/place//data=!4m2!3m1!1s0x80c2c778e3b73d33:0xbdc58662a4a97d49?authuser=-1&hl=en&gl=us'}

De los datos obtenidos anteriormente nos quedaremos con `gmap_id` cuya información coincide con los de la segunda carpeta `reviews-estados`. En la carpeta reviews estados se incluyen otras 51 carpetas que contienen la información de cada estado, a su vez la carpeta de cada estado contiene varios archivos JSON con las reviews.

Para hacer más fácil la carga de todos estos datos a Google Cloud Platform, filtraremos todos estos archivos por las categorías de restaurantes y hoteles. Para ello usaremos el campo `gmap_id` del archivo que cargamos anteriormente. Al estar filtrado ya este archivo solo debemos buscar los códigos de `gmap_id` en los archivos de reviews por estado que se incluyan en la siguiente lista.

In [ ]:
lista_gmap_id = []
for registro in business:
  lista_gmap_id.append(registro['gmap_id'])

Para filtrar y juntar todos los archivos de cada estado y obtener uno solo usaremos el código a continuación que automatiza gran parte del proceso accediendo a la carpeta de cada estado para filtrar y agregar cada archivo. Adicionalmente agrega el campo “estado”.

Debido al costo computacional se ejecuta el código con los reviews de 17 estados y se van cargando a un archivo json.

In [ ]:
lista_gmap_id_set = set(lista_gmap_id)

# Ruta de la carpeta "reviews-estados" en Google Drive
ruta_carpeta_estados = '/content/drive/MyDrive/Colab Notebooks/Google Maps/reviews-estados'

# Función para combinar los archivos JSON de un estado
def combinar_archivos_json(carpeta_estado):
    ruta_carpeta_estado = os.path.join(ruta_carpeta_estados, carpeta_estado)
    data_estado = []
    for archivo in os.listdir(ruta_carpeta_estado):
        if archivo.endswith('.json'):
            ruta_archivo = os.path.join(ruta_carpeta_estado, archivo)
            with open(ruta_archivo, 'r') as file:
                contenido = [json.loads(linea) for linea in file]
                sin_duplicados = list(OrderedDict((str(d), d) for d in contenido).values())
                for registro in sin_duplicados:
                  if registro['gmap_id'] in lista_gmap_id_set:
                      data_estado.append(registro)
    return data_estado

# Combinar los archivos JSON de cada estado y agregar el campo de estado
datos_totales = []
for estado in os.listdir(ruta_carpeta_estados)[:17]:
    datos_estado = combinar_archivos_json(estado)
    for registro in datos_estado:
        registro['estado'] = estado
    datos_totales.extend(datos_estado)

In [ ]:
# Convierte la lista de objetos JSON en un archivo JSONL

with open('/content/drive/MyDrive/reviews_estados_3.jsonl', 'w') as jsonl_file:
    for item in datos_totales:
        jsonl_file.write(json.dumps(item) + '\n')

In [ ]:
len(datos_totales)

3661175